# Handling Large Data Sets
Until now, we have applied our framework to a very small simulation. However, what if we are working with a very large data set (like TNG50-1, which has $2160^3$ particles, $512$ times more than TNG50-4)?

# Starting simple: computing in chunks

First, we can still run the same calculation as above, and it will "just work" (hopefully).

This is because Dask has many versions of common algorithms and functions which work on "blocks" or "chunks" of the data, which split up the large array into smaller arrays. Work is needed on each chunk, after which the final answer is assembled.

Importantly, in our case above, even if the `mass` array above does not fit into memory, the `mass.sum().compute()` will chunk the operation up in a way that the task can be calculated.

In [1]:
from astrodask import load
ds = load("/data/public/astrodask-testdata/TNG50-3_snapshot")

INFO: /Header NumPart_ThisFile has different values.
/tmp/tmp9p_a4lqv.hdf5 <HDF5 file "tmp9p_a4lqv.hdf5" (mode r)>
<HDF5 file "tmp9p_a4lqv.hdf5" (mode r)>


Before we start, let's enable a progress indicator from dask (note that this will only work for local schedulers, see next section):

In [2]:
from dask.diagnostics import ProgressBar
ProgressBar().register()

And then we can request the actual computation:

In [3]:
%time ds.data["PartType0"]["Masses"].sum().compute()

[########################################] | 100% Completed | 304.18 ms
CPU times: user 52.1 ms, sys: 263 ms, total: 315 ms
Wall time: 309 ms


518363.12

While the result is eventually computed, it is a bit slow, primarily because the actual reading of the data off disk is the limiting factor, and we can only use resources available on our local machine.

## More advanced: computing in parallel

Rather than sequentially calculating large tasks, we can also run the computation in parallel. 

To do so different advanced dask schedulers are available. Here, we use the most straight forward [distributed scheduler](https://docs.dask.org/en/latest/how-to/deploy-dask/single-distributed.html).

Usually, we would start a scheduler and then connect new workers (e.g. running on multiple compute/backend nodes of a HPC cluster). After, tasks (either interactively or scripted) can leverage the power of these connected resources.

For this example, we will use the same "distributed" scheduler/API, but keep things simple by using just the one (local) node we are currently running on.

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1, 
                       dashboard_address=":8787")
client = Client(cluster)

2023-01-09 15:11:46,879 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jjd_0_t_', purging


Here is our client. We can access the scheduler on specified dashboard port to investigate its state.

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 47.05 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42495,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 47.05 GiB
Comm: tcp://127.0.0.1:43743,Total threads: 1
Dashboard: http://127.0.0.1:46275/status,Memory: 5.88 GiB
Nanny: tcp://127.0.0.1:44527,


We can now perform the same operations, but it is performed in a distributed manner, in parallel.

One significant advantage is that (even when using only a single node) individual workers will load just the subsets of data they need to work on, meaing that I/O operations become parallel.

Note: after creating a `Client()`, all calls to `.compute()` will automatically use this scheduler and its set of workers.

In [6]:
%time ds.data["PartType0"]["Masses"].sum().compute()

CPU times: user 79 ms, sys: 44 ms, total: 123 ms
Wall time: 581 ms


518363.12

The progress bar, we could use for the default scheduler (before initializing `LocalCluster`), is unavailable for the distributed scheduler. However, we can still view the progress of this task as it executes using its status dashboard (as a webpage in a new browser tab or within [jupyter lab](https://github.com/dask/dask-labextension)). You can find it by clicking on the "Dashboard" link above. If running this notebook server remotely, e.g. on a login node of a HPC cluster, you may have to change the '127.0.0.1' part of the address to be the same machine name/IP.